In [1]:
# Lo primero sera importar las librerias y estableces algunas configuraciones para
# facilitar la lectura en pantalla

import pandas as pd
import numpy as np
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

In [8]:
# leer los datos y cargarlos en dataframe, validando la carga al presentar informacion basica
# del archivo
mensajes = pd.read_json('C:/Users/Luis/Documents/Ciencia de Datos/Dataset1/mensajespd.json')
print(mensajes.head())
print("\n")
print("Cantidad de registros de mensajes = %3d" % mensajes.shape[0])
print("\n")
print(mensajes.columns)

   Enviado/por           Fecha/Hora  \
0  04140819214  29/08/2018 16:31:19   
1  04140819214  30/08/2018 08:34:25   
2  04145328570  28/08/2018 18:03:12   
3  04145328570  28/08/2018 18:03:19   
4  04145328570  01/09/2018 10:15:48   

                                                                                                                                                          Texto  
0  Luis que si le mando la plata de bicentimario 600 bolibares soberanos ala cuenta mercantil                                                                    
1  ps luis qe si mamdo la plata que esta en mi cuenta vecentenario a la mercantil                                                                                
2  Hola Dios te bendiga                                                                                                                                          
3  No hay internet                                                                                                   

In [9]:
# ahora se va a proceder a colocar en minuscula, todo el contenido de la variable "Texto"
# todo a minuscula

mensajes.Texto = mensajes.Texto.str.lower()

# y luego eliminamos todos los acentos 

import re
import unicodedata

def strip_accents(text):
    """
    Strip accents from input String.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

mensajes["Texto"] = mensajes["Texto"].apply(lambda tx: strip_accents(tx))



**el dataset requiere depuracion ya que multiples
 registros (algunos), contienen informacion de un solo mensaje, lo que distorciona la
 interpretacion de dichos mensajes
 primer paso ordenar el dataframe por "Enviado/por" y "Fecha/Hora"**

In [10]:
# ordenamos
mensajes.sort_values(by=["Enviado/por", "Fecha/Hora"])

# convertimos a formato de fecha la columna "Fecha/Hora"
mensajes["Fecha/Hora"] = pd.to_datetime(mensajes["Fecha/Hora"])

# variable de interes tiempo en segundos (hora del dia en segundos)

mensajes['ts'] = mensajes["Fecha/Hora"].apply(lambda x: x.hour*60*60+x.minute*60+x.second)

# el tiempo entre mensajes: tem
# voala! funciona.. 

mensajes["tem"] = np.zeros(mensajes.shape[0], dtype=int)

mensajes["tem"] = abs(mensajes["ts"].diff())


# con este arreglo de indice calculado a continuacion, concatenamos 
# mensajes divididos en la fila i-1 y posterior eliminamos los (i) ndice
# ya q seran redundantes
# hay q tener en cuenta filtrar q mensajes a concatenar sean del mismo
# sender y que contenido de mensajes sean distintos

# cosideramos solo aquellos mensajes sucesivos que hayan sido recibidos
# en un intervalo de tiempo menor a 10 segundos ya que en este subgrupo se encuentran
# los mensajes divididos

import warnings
warnings.filterwarnings('ignore')

indimulmensajes = mensajes[mensajes["tem"]<10].index

# uniendo
import operator
for i in reversed(indimulmensajes):
    if operator.and_(mensajes["Enviado/por"][i-1]==mensajes["Enviado/por"][i], mensajes["Texto"][i-1]!=mensajes["Texto"][i]):
        mensajes["Texto"][i-1] += mensajes["Texto"][i]

# borrando redundancia
mensajes.drop(mensajes.index[indimulmensajes], inplace = True)


In [11]:
print(mensajes.shape)
print(mensajes.columns)
print(mensajes[mensajes["tem"]<10].count())

(1016, 5)
Index(['Enviado/por', 'Fecha/Hora', 'Texto', 'ts', 'tem'], dtype='object')
Enviado/por    0
Fecha/Hora     0
Texto          0
ts             0
tem            0
dtype: int64


**es_suscriptor: esta variable toma valor 0 si el contenido de "Texto" proviene de un
cliente de la red comunitaria, y 1 en caso contrario. Pero ¿Como se determina si el 
mensaje proviene o no de un suscriptor?**
**a vuelo de pajaro podemos decir que:**

**1- Si hacemos query de "Enviado/por" en la base de datos de cliente y aparece en esta
definitivamente el mensaje fue enviado por un suscriptor, es el caso mas sencillo**

**2- Que pasa si el query no devuelve datos, quiere decir que no se encuentra registrado
pero esto no quiere decir que el mensaje no provenga de un cliente, sino que tal vez
este usando otro dispositivo no registrado en la base de datos, de alli que surga la necesidad
de analizar el contenido para establecer el valor de la variable es_suscriptor.
en este apartado se puede inferir que tal vez el usuario se identifique con su nombre
dentro del contenido del mensaje o tal vez no, en caso de que si, la solucion pasaria algo
similar a 1, en caso contrario se necesitara establecer mas precision en cuanto al contenido
de "Texto" y hasta sera necesario interpretar varios mensajes simultaneamente del mismo 
Enviado/por" en un tiempo-estipulado e interactuar con el remintente para clasificarlo.**

**3- Otro aspecto que pareciera ser prometedor es la frecuencia de origen de sms, obtenida de 
la variable "Enviado/por"**


In [55]:
# creando 2 variables, una que indique si "Texto" proviene de un suscriptor del servicio
# de internet (0) o no (1) y otra que partiendo del contenido de "Texto" clasifique en mensaje,
# lo cual se hara luego de analizar la variable "Texto" mas adelante

# por defecto ninguno es suscriptor
mensajes["es_suscriptor"] = np.ones(mensajes.shape[0], dtype=int)

# aun no se a establecido criterio para clasificar, se inicializa en 0
mensajes["tipo_mensaje"] = np.zeros(mensajes.shape[0], dtype=int)

In [56]:
# leer telefonos conocidos de clientes desde archivo
# al archivo tlfclientes.csv se le agrego un caracter alfanumerico al comienzo
# de cada registro para que pandas leyera como texto y no como numero

telef_clientes = pd.read_csv('C:/Users/Luis/Documents/Ciencia de Datos/Dataset1/tlfclientes.csv')
print(telef_clientes.shape)
print(telef_clientes.head())
print(telef_clientes.columns)

# jugando un poco con los dataframe, obteniendo salidas de datos relacionadas
#print(mensajes["Enviado/por"][4])

#print(telef_clientes.telef[37][1:-1])

#print(telef_clientes["telef"][1:-1])

#print(telef_clientes[telef_clientes.telef==mensajes["Enviado/por"][4]])

#print(mensajes.Texto[mensajes["Enviado/por"]==telef_clientes.telef[37][1:-1]])


# ahora se convierte telef_clientes.telef eliminando los caracteres- al inicio y final
telef_clientes["telef"] = telef_clientes["telef"].str.replace("-","")

# y se carga a memoria como lista, ya q son pocos
lista_telef = telef_clientes["telef"].tolist()

print(len(lista_telef))

                                          

(120, 1)
           telef
0  -04147062906-
1  -04247807722-
2  -04247657429-
3  -04120713877-
4  -04266670618-
Index(['telef'], dtype='object')
120


In [57]:
# ahora clasificamos como 0 la variable "es_suscriptor", si coincide lista con 
# "Enviado/por" del dataframe

# print(mensajes.Texto[mensajes["Enviado/por"].isin(lista_telef)])

mensajes.loc[mensajes["Enviado/por"].isin(lista_telef), "es_suscriptor"] = 0 

# validamos con un crosstab, bien ya tenemos parte de los mensajes clasificados

tab = pd.crosstab(index=mensajes["es_suscriptor"],columns="frecuencia")
print(tab)
print("\n", "conteo")
#print(mensajes["es_suscriptor"].value_counts())
print(mensajes.es_suscriptor.value_counts())



col_0          frecuencia
es_suscriptor            
0              254       
1              762       

 conteo
1    762
0    254
Name: es_suscriptor, dtype: int64



**Bien, repasando de nuevo, un mensaje se considera que proviene de un suscriptor en estos 
casos:**

**1- su origen plasmado en la variable "Enviado/por", resuelto anteriormente**

**2- contenga frases asociadas con el servicio de internet, en especial las siguientes
son trascendentes: "no hay conexion", "no hay internet", "cuanto es la mensualidad"**

**3- tomar en cuenta comodin por error ortografico comun "interne" y otros


In [60]:
# palabras de interes dado el contexto en que se esta trabajando:
# 1 interne (comodin de internet):conexion

mensajes["internet"] = np.ones(mensajes.shape[0], dtype=int)
mensajes.loc[mensajes['Texto'].str.contains('interne'), "internet"] = 0 
print(mensajes.internet.value_counts())
print(pd.crosstab(mensajes["internet"], columns=mensajes["es_suscriptor"]))

mensajes["conexion"] = np.ones(mensajes.shape[0], dtype=int)
mensajes.loc[mensajes['Texto'].str.contains('conexion'), "conexion"] = 0 
print(mensajes.internet.value_counts())
print(pd.crosstab(mensajes["conexion"], columns=mensajes["es_suscriptor"]))


# 1 mes o mensualidad (comodin de "cuanto"):
mensajes["cuanto"] = np.ones(mensajes.shape[0], dtype=int)
mensajes.loc[mensajes['Texto'].str.contains('cuanto'), "cuanto"] = 0 
print(mensajes.internet.value_counts())
print(pd.crosstab(mensajes["cuanto"], columns=mensajes["es_suscriptor"]))




1    703
0    313
Name: internet, dtype: int64
es_suscriptor    0    1
internet               
0              130  183
1              124  579
1    703
0    313
Name: internet, dtype: int64
es_suscriptor    0    1
conexion               
0              19   18 
1              235  744
1    703
0    313
Name: internet, dtype: int64
es_suscriptor    0    1
cuanto                 
0              17   46 
1              237  716


In [7]:
# listando un subgrupo de interes a partir de la tabulacion cruzada anterior

import operator

mensajes.loc[operator.and_(mensajes.es_suscriptor == 0, mensajes.internet == 1)]


,Enviado/por,Fecha/Hora,Texto,es_suscriptor,tipo_mensaje,internet
2,04145328570,28/08/2018 18:03:12,hola dios te bendiga,0,0,1
5,04145328570,01/09/2018 10:16:09,puede verificar por mi numero de cédula 9791646,0,0,1
6,04145328570,01/09/2018 10:16:14,es del provincial,0,0,1
11,04241029944,29/08/2018 10:45:42,le varios meses...,0,0,1
12,04241029944,29/08/2018 10:57:08,"ya le transferi 18 por dinero rapido de provincial, ya lo debe tener en su cuenta, gracias",0,0,1
37,04164451445,03/09/2018 06:56:45,red no identificada sin acceso a interne,0,0,1
39,04247308368,02/09/2018 13:21:15,donde esta para pagar el interne,0,0,1
61,04147460929,03/09/2018 12:55:13,cuanto es la mensualidad don luis,0,0,1
64,04164451445,03/09/2018 06:56:45,red no identificada sin acceso a interne,0,0,1
68,04247308368,02/09/2018 13:21:15,donde esta para pagar el interne,0,0,1
